In [1]:
import requests
from bs4 import BeautifulSoup
import os
import re
from dotenv import load_dotenv

load_dotenv("C:/Users/Jochen/THWSkraken/credentials.env")

url = 'https://elearning.fhws.de/login/index.php'
values = {'username': os.environ.get('STUDENT_USER'),
          'password': os.environ.get('STUDENT_PASSWORD'),
          'anchor': ''}


In [3]:
sess = requests.Session()

In [7]:
def getCRSFToken():
    r = sess.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup.find('input', {'name': 'logintoken'})['value']

def login():
    values['logintoken'] = getCRSFToken()
    r = sess.post(url, data=values)
    print("login:", r.status_code)
    return r

def getCourses():
    index = 0
    ajax1 = "https://elearning.fhws.de/theme/remui/request_handler.php?action=get_courses_ajax&wdmdata={'category':'all','sort':null,'search':'','tab':true,'page':{'courses':0,'mycourses':"
    ajax2 = "},'pagination':true,'view':null,'isFilterModified':true}"
    courses = []
    r = sess.get(ajax1 + str(index) + ajax2)
    print("get courses:", r.status_code)
    content = r.json()
    maxIndex = max(map(int, re.findall("page=(\d)", content["pagination"])))
    courses.append(content["courses"])
    index += 1
    while index <= maxIndex:
        r = sess.get(ajax1 + str(index) + ajax2)
        content = r.json()
        courses.append(content["courses"])
        index += 1
    return courses

In [6]:
login()

login: 200


<Response [200]>

In [ ]:
courses = getCourses

In [3]:
page = sess.get(url)
print(page.cookies)
signin = BeautifulSoup(page.content, 'html.parser')
token = signin.find('input', {'name': 'logintoken'})['value']
print(token)
values['logintoken'] = token
r = requests.post(url, data=values, headers=dict(Referer=url), cookies=page.cookies)
print(r.status_code)
print(r.cookies)
print(r.history[0].cookies)
cookies = r.history[0].cookies
r = requests.get(
    'https://elearning.fhws.de/theme/remui/request_handler.php?action=get_courses_ajax&wdmdata={'category":"all","sort":null,"search":"","tab":true,"page":{"courses":0,"mycourses":0},"pagination":true,"view":null,"isFilterModified":true}',
    headers=dict(Referer='https://elearning.fhws.de/course/index.php?mycourses=1'), cookies=cookies)
print(r.status_code)
content = r.json()
print(content.keys())
print(len(content['courses']))
print(content['courses'][0])


<RequestsCookieJar[<Cookie MoodleSession=c6k9rkithv5nm4876h7ru011gj for elearning.fhws.de/>]>
aHP05LDKBB7ZvUwDrETWcACnt1NMPrB8
200
<RequestsCookieJar[]>
<RequestsCookieJar[<Cookie MoodleSession=2dpqf248k7q1cqgrgppmgp91vj for elearning.fhws.de/>]>
200
dict_keys(['pagination', 'courses', 'view'])
12
{'id': '9810', 'grader': 'https://elearning.fhws.de/grade/report/grader/index.php?id=9810', 'shortname': '<div class="text_to_html">Fix_AWPF_GrFH</div>', 'courseurl': 'https://elearning.fhws.de/course/view.php?id=9810', 'coursename': '<div class="text_to_html">Gründen@FHWS</div>', 'enrollusers': 'https://elearning.fhws.de/user/index.php?id=9810&version=9810', 'editcourse': 'https://elearning.fhws.de/course/edit.php?id=9810', 'categoryname': '<div class="text_to_html">AWPF-Fix</div>', 'courseimage': 'https://elearning.fhws.de/pluginfile.php/782757/course/overviewfiles/200812_ENTREPRENEUR_Logo_kompakt.jpg', 'coursesummary': '', 'startdate': {'day': '01', 'month': 'Oct', 'year': '2021'}, 'mycour

In [4]:
content["pagination"]

'<nav aria-label="Seite">\n        <ul class="pagination pagination-no-border">\n                <li class="page-item active disableClick">\n                    <a href="" class="page-link mx-sm-5">\n                        1\n                            <span class="sr-only">(aktuell)</span>\n                    </a>\n                </li>\n                <li class="page-item ">\n                    <a href="javascript://void(0);?page=1" class="page-link mx-sm-5">\n                        2\n                    </a>\n                </li>\n                <li class="page-item ">\n                    <a href="javascript://void(0);?page=2" class="page-link mx-sm-5">\n                        3\n                    </a>\n                </li>\n                <li class="page-item ">\n                    <a href="javascript://void(0);?page=3" class="page-link mx-sm-5">\n                        4\n                    </a>\n                </li>\n                <li class="page-item">\n    

In [5]:
from kraken import SoupChef
sou = SoupChef()

In [8]:
soup = sou.get_soup_from_text(content["pagination"])

In [13]:
results = soup.find_all("a")
results[0]

<a class="page-link mx-sm-5" href="">
                        1
                            <span class="sr-only">(aktuell)</span>
</a>